In [1]:
from selenium import webdriver
import time
from bs4 import BeautifulSoup
import pandas as pd

### 계정정보추출

In [2]:
driver = webdriver.Chrome('c:/chromedriver/chromedriver.exe')
url = 'http://fifaonline4.nexon.com/datacenter/rank'
driver.get(url)

In [3]:
def go_nextpage_10():
    next = driver.find_element_by_css_selector('#middle > div > div > div.board_list.datacenter_rank > div.pagination > div > a.btn_next_list.ajaxNav')
    next.click()
    time.sleep(1)

def go_nextpage(i):
    next = driver.find_element_by_css_selector('#middle > div > div > div.board_list.datacenter_rank > div.pagination > div > ul > li:nth-child('+str(i)+') > a')
    next.click()
    time.sleep(1)

def get_ranker():
    html = driver.page_source
    soup = BeautifulSoup(html,'lxml')
    number=[]; name=[]
    
    for i in list(range(1,21)):
        tmp = soup.select('#middle > div > div > div.board_list.datacenter_rank > div.content.datacenter_rank_list > div > div.tbody > div:nth-child('+str(i)+') > span.td.rank_coach > span.coach_wrap > span.name.profile_pointer')
        data = str(tmp[0])
        number.append(data[data.index('n="')+3 : data.index('">')])
        name.append(data[data.index('">')+2 : data.index('</')])
    return number, name

In [4]:
ranker_number = []
ranker_name = []

#1위~200위

a,b = get_ranker()
ranker_number = ranker_number + a
ranker_name = ranker_name + b

for i in list(range(2,11)):
    go_nextpage(i)
    a,b = get_ranker()

    ranker_number = ranker_number + a
    ranker_name = ranker_name + b

#201위~10000위
for i in range(49):
    go_nextpage_10()
    
    a,b = get_ranker()
    ranker_number = ranker_number + a
    ranker_name = ranker_name + b

    for i in list(range(2,11)):
        go_nextpage(i)
        a,b = get_ranker()

        ranker_number = ranker_number + a
        ranker_name = ranker_name + b

In [6]:
ranker[ranker.duplicated()==True]

,이름,계정번호


In [14]:
ranker = {'순위':list(range(1,10001)),'이름':ranker_name,'계정번호':ranker_number}
ranker = pd.DataFrame(ranker)
ranker = ranker.set_index('순위')
ranker['LIV']='n'
ranker.to_csv('ranker.txt')

### 리버풀유저추출

In [4]:
driver = webdriver.Chrome('c:/chromedriver/chromedriver.exe')

In [17]:
ranker = pd.read_csv('ranker.txt')

In [8]:
ranker

,순위,이름,계정번호,Liv
0,1,20010124,1896137692,n
1,2,언베리환경,151208091,n
2,3,AF유정훈,1795254104,n
3,4,SB차현우,1376045793,n
4,5,T1SeYun,1224946369,n
...,...,...,...,...
9995,9996,Alex스타즈,671472071,n
9996,9997,Samuel움티티,252166017,n
9997,9998,붐붐차차차차,855824193,n
9998,9999,우리흥토튼햄,453626383,n


In [105]:
for i in list(range(999,2000)):
    url = 'http://fifaonline4.nexon.com/profile/squad/popup/'+str(ranker['계정번호'][i])
    driver.get(url)
    time.sleep(1.5)
    
    html = driver.page_source
    soup = BeautifulSoup(html,'lxml')
    skip = soup.select('#squadMaker > div.squadmaker-wrap.squad__wrap > div.squadmaker-aside > div.squadmaker-aside__side-wrap > div.squadmaker-aside__teamcolor > a.btn_teamcolor.team1.disable > div > div.teamcolor_item_list')
    if skip != []:
        continue
    tmp = soup.select('#squadMaker > div.squadmaker-wrap.squad__wrap > div.squadmaker-aside > div.squadmaker-aside__side-wrap > div.squadmaker-aside__teamcolor > a.btn_teamcolor.team1 > div > div.teamcolor_item_list > div:nth-child(1) > div.teamcolor_item_tit > div.teamcolor_item_add_item_tit_txt')
    tmp = tmp[0].text
    if tmp == '리버풀':
        liv = soup.select('#squadMaker > div.squadmaker-wrap.squad__wrap > div.squadmaker-aside > div.squadmaker-aside__side-wrap > div.squadmaker-aside__teamcolor > a.btn_teamcolor.team1 > div > div.teamcolor_item_list > div:nth-child(1) > div.teamcolor_item_tit > div.teamcolor_item_add_item_tit_num')[0].text[:-1]
        if int(liv) >= 9:
            ranker['LIV'][i]='y'

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


In [67]:
html = driver.page_source
soup = BeautifulSoup(html,'lxml')

In [120]:
ranker_LIV = ranker[ranker['LIV']=='y']
ranker_LIV

,순위,이름,계정번호,LIV
582,583,짱다이크,470008787,y
722,723,위르겐클롭구단,67262470,y
727,728,삐에루,1694912968,y
916,917,사당신선,1141156740,y
944,945,CaptainHumba,553689693,y
968,969,언베리버풀,772159631,y
974,975,너정체가뭐냐,1409780347,y
976,977,Firmino,822112180,y
994,995,Virgil반데이크,503778625,y
1126,1127,아브다비,83956906,y


In [121]:
ranker_LIV = ranker_LIV.reset_index(drop=True)
ranker_LIV

,순위,이름,계정번호,LIV
0,583,짱다이크,470008787,y
1,723,위르겐클롭구단,67262470,y
2,728,삐에루,1694912968,y
3,917,사당신선,1141156740,y
4,945,CaptainHumba,553689693,y
5,969,언베리버풀,772159631,y
6,975,너정체가뭐냐,1409780347,y
7,977,Firmino,822112180,y
8,995,Virgil반데이크,503778625,y
9,1127,아브다비,83956906,y


In [108]:
ranker_LIV.to_csv('ranker_LIV.txt')

### 스쿼드 자동확인

In [123]:
for num in ranker_LIV['계정번호']:
    url = 'http://fifaonline4.nexon.com/profile/squad/popup/'+str(num)
    driver.get(url)
    time.sleep(10)

In [122]:
ranker_LIV['실축전술']='n'
ranker_LIV

,순위,이름,계정번호,LIV,실축전술
0,583,짱다이크,470008787,y,n
1,723,위르겐클롭구단,67262470,y,n
2,728,삐에루,1694912968,y,n
3,917,사당신선,1141156740,y,n
4,945,CaptainHumba,553689693,y,n
5,969,언베리버풀,772159631,y,n
6,975,너정체가뭐냐,1409780347,y,n
7,977,Firmino,822112180,y,n
8,995,Virgil반데이크,503778625,y,n
9,1127,아브다비,83956906,y,n


In [126]:
ranker_LIV['실축전술'][1,5,6,7,9,10,11,13,14,16,17,18,19,20]='y'
ranker_LIV

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


,순위,이름,계정번호,LIV,실축전술
0,583,짱다이크,470008787,y,n
1,723,위르겐클롭구단,67262470,y,y
2,728,삐에루,1694912968,y,n
3,917,사당신선,1141156740,y,n
4,945,CaptainHumba,553689693,y,n
5,969,언베리버풀,772159631,y,y
6,975,너정체가뭐냐,1409780347,y,y
7,977,Firmino,822112180,y,y
8,995,Virgil반데이크,503778625,y,n
9,1127,아브다비,83956906,y,y


In [127]:
ranker_LIV[ranker_LIV['실축전술']=='y']

,순위,이름,계정번호,LIV,실축전술
1,723,위르겐클롭구단,67262470,y,y
5,969,언베리버풀,772159631,y,y
6,975,너정체가뭐냐,1409780347,y,y
7,977,Firmino,822112180,y,y
9,1127,아브다비,83956906,y,y
10,1446,쥐찡,1845924977,y,y
11,1538,se7en애플망고,1057420604,y,y
13,1602,Halftime두두,151589699,y,y
14,1644,DTLIV,2130931827,y,y
16,1680,BBobki,973099148,y,y


In [128]:
ranker_LIV.to_csv('ranker_LIV.txt')